# Automatic Sentiment Analysis
### Individual Assignment for 5508
### HE Peiwei, 54471952

# 1 Introduction
This report is to use different models to do the sentiment analysis with the Amazon reviews data. The dataset is Digital Music reviews. I defines the reviews which scores over 4 as good products, and seperate the data into 3 sets, 80% as training set, 15% as evaluation set and 15% as testing set.

In [1]:
import numpy as np
import pandas as pd
from sklearn import metrics

In [2]:
#Load the data and define the categories
reviews = pd.read_pickle("reviews.pickle")
products = pd.read_pickle("products.pickle")

In [40]:
reviews['good'] = reviews.overall > 4
#80% as training, 15% as evaluation, 15% as testing
train = reviews.sample(frac=.7)
evaluation = reviews.drop(train.index).sample(frac=.5)
testing = reviews.drop(train.index).drop(evaluation.index)
reviews.head(2)

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,good
0,5555991584,"[3, 3]",5.0,"It's hard to believe ""Memory of Trees"" came ou...","09 12, 2006",A3EBHHCZO6V2A4,"Amaranth ""music fan""",Enya's last great album,1158019200,True
1,5555991584,"[0, 0]",5.0,"A clasically-styled and introverted album, Mem...","06 3, 2001",AZPWAXJG9OJXV,bethtexas,Enya at her most elegant,991526400,True


# 2 Model Building
## 2.1 Baseline Model
Firstly, build a baseline moedel which simply assumes that the most frequent label will always be predicted.

In [4]:
# Predict everything by the most frequently categories
preditions_most=train.good.value_counts().idxmax()
predictions_baseline=np.array(preditions_most).repeat(len(evaluation))

In [5]:
print(pd.crosstab(predictions_baseline, evaluation.good, rownames=['predictions_baseline'], colnames=['real']))
print(metrics.classification_report(evaluation.good, predictions_baseline, target_names=["Bad", "Good"]))

real                  False  True 
predictions_baseline              
True                   4340   5366
             precision    recall  f1-score   support

        Bad       0.00      0.00      0.00      4340
       Good       0.55      1.00      0.71      5366

avg / total       0.31      0.55      0.39      9706



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


As the result shown, the precision of this model by applying evaluation data is very low. And baseline model could only predict the good review as the scores for the bad reviews are zero.

## 2.2 Initial Model
Create a DTM of the review text, then use TF-IDF weighting to evaluate the importance of document, and use Naive Bayes model as the classifier.

In [42]:
# Make the DTM  
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
dtm_train = count_vect.fit_transform(train.reviewText)
dtm_evaluation = count_vect.transform(evaluation.reviewText)
dtm_testing =count_vect.transform(testing.reviewText)

In [43]:
#use TF-IDF weighting
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=True)
dtm_train = tf_transformer.fit_transform(dtm_train)
dtm_evaluation = tf_transformer.transform(dtm_evaluation)
dtm_testing = tf_transformer.transform(dtm_testing)

In [8]:
# Train NB model
from sklearn.naive_bayes import MultinomialNB
clf_NB = MultinomialNB().fit(dtm_train, train.good)

In [9]:
# Predict the categories
predictions_initial = clf_NB.predict(dtm_evaluation)
predictions_initial_testing = clf_NB.predict(dtm_testing)

In [10]:
print("Below are the evaluation set result of initial model")
print(pd.crosstab(predictions_initial, evaluation.good, rownames=['evaluation_set'], colnames=['real']))
print(metrics.classification_report(evaluation.good, predictions_initial, target_names=["Bad", "Good"]))

Below are the evaluation set result of initial model
real            False  True 
evaluation_set              
False            1686    395
True             2654   4971
             precision    recall  f1-score   support

        Bad       0.81      0.39      0.53      4340
       Good       0.65      0.93      0.77      5366

avg / total       0.72      0.69      0.66      9706



In [11]:
print("Below are the testing set result of initial model")
print(pd.crosstab(predictions_initial_testing, testing.good, rownames=['testing_set'], colnames=['real']))
print(metrics.classification_report(testing.good, predictions_initial_testing, target_names=["Bad", "Good"]))

Below are the testing set result of initial model
real         False  True 
testing_set              
False         1763    383
True          2596   4964
             precision    recall  f1-score   support

        Bad       0.82      0.40      0.54      4359
       Good       0.66      0.93      0.77      5347

avg / total       0.73      0.69      0.67      9706



As the result shown, the precision of bad reviews is 0.81, while the precision of good ones is 0.64, which means the Naive Bayes model could predict the bad reviews more precise than the good reviews, and the average performance is obvious better than the baseline model, as the precision is 0.72, f1-score is 0.65.

## 2.3 Improved Model
### 2.3.1 Feature Dimension Reduction
I've tried different classifiers such as KNN, SVM for the improvement, however the output is not very good. So I continue to do some improvement on the previous moedel. To improve the initial model, I conduct the improvement on the DTM, by reducing its dimensionality. After improved the DTM, I apply the same Navie Bayes model to predict the review text.

In [12]:
#Feature dimension reduction
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
ch2 = SelectKBest(chi2, k=1800)
dtm_train = ch2.fit_transform(dtm_train, train.good)
dtm_evaluation = ch2.transform(dtm_evaluation)
dtm_testing = ch2.transform(dtm_testing)

In [13]:
# Train NB model
from sklearn.naive_bayes import MultinomialNB
clf_NB = MultinomialNB(alpha=1.2,fit_prior=False).fit(dtm_train, train.good)

In [14]:
# Predict the categories
predictions_improved = clf_NB.predict(dtm_evaluation)
predictions_improved_testing = clf_NB.predict(dtm_testing)

In [15]:
# Show the results
print("Below are the evaluation set result of improved NB model")
print(pd.crosstab(predictions_improved, evaluation.good, rownames=['evaluation_set'], colnames=['real']))
print(metrics.classification_report(evaluation.good, predictions_improved, target_names=["Bad", "Good"]))

Below are the evaluation set result of improved NB model
real            False  True 
evaluation_set              
False            2929   1288
True             1411   4078
             precision    recall  f1-score   support

        Bad       0.69      0.67      0.68      4340
       Good       0.74      0.76      0.75      5366

avg / total       0.72      0.72      0.72      9706



In [16]:
print("Below are the testing set result of improved NB model")
print(pd.crosstab(predictions_improved_testing, testing.good, rownames=['testing_set'], colnames=['real']))
print(metrics.classification_report(testing.good, predictions_improved_testing, target_names=["Bad", "Good"]))

Below are the testing set result of improved NB model
real         False  True 
testing_set              
False         3041   1249
True          1318   4098
             precision    recall  f1-score   support

        Bad       0.71      0.70      0.70      4359
       Good       0.76      0.77      0.76      5347

avg / total       0.74      0.74      0.74      9706



As the result shown, the difference of the precisions on bad and good reviews has been reduced, while the overall precision has been improved to 0.72, and the f1-score is 0.72, which means feature dimension reduction is effective for improving the initial model.

### 2.3.2 SVM - Grid Search
I use grid search to find the best parameters for SVM model.

In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm  

In [19]:
# set the parameters 
parameters = {'alpha': (0.00005,0.0001,0.001),
              'average': (True, False),
              'epsilon': (0.1, 0.2),
}

In [20]:
#set the classifer
from sklearn.linear_model import SGDClassifier
clf_SVM_gs=SGDClassifier(loss='hinge',alpha=5e-05,learning_rate='optimal',class_weight={1: 1.2})
gs_clf = GridSearchCV(clf_SVM_gs, parameters, n_jobs=-1)

In [21]:
#train the classifer
gs_clf = gs_clf.fit(dtm_train,train.good)
predictions_gs = gs_clf.predict(dtm_evaluation)

In [22]:
print("The best error rate：%r" % (gs_clf.best_score_))
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))
print(pd.crosstab(predictions_gs, evaluation.good, rownames=['predictions_gs'], colnames=['real']))
print(metrics.classification_report(evaluation.good, predictions_gs, target_names=["Bad", "Good"]))

The best error rate：0.75762794189075811
alpha: 5e-05
average: True
epsilon: 0.2
real            False  True 
predictions_gs              
False            2657    729
True             1683   4637
             precision    recall  f1-score   support

        Bad       0.78      0.61      0.69      4340
       Good       0.73      0.86      0.79      5366

avg / total       0.76      0.75      0.75      9706



As the result shown, the best parameter for the SVM model is alpha=5e-05, so I apply this value to the model, fortunately, the precision seems to be a little higher than the improved naive bayes model, which is 0.75, while the f1-score is 0.75.

### 2.3.3 Personalized Prediction
To personalize the prediction by using characteristics of the reviewer, I use the average score of the reviews of each reviewer and average score of each product as new feature, so as to predict the good and bad reviews by using SVM as the classifier.

In [31]:
#compute the average score of the reviews of each reviewer on training set and apply to the whole data set
index_average_reviewers=(pd.DataFrame(train.groupby('reviewerID').overall.mean()))
index_average_reviewers.columns = ['average_reviewers']
train=train.join(index_average_reviewers, on='reviewerID')
evaluation=evaluation.join(index_average_reviewers, on='reviewerID')
testing=testing.join(index_average_reviewers, on='reviewerID')

In [32]:
#compute the average score of the product of each product on training set and apply to the whole data set
index_average_asin=(pd.DataFrame(train.groupby('asin').overall.mean()))
index_average_asin.columns = ['average_asin']
train=train.join(index_average_asin, on='asin')
evaluation=evaluation.join(index_average_asin, on='asin')
testing=testing.join(index_average_asin, on='asin')
train.head(1)

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,good,average_reviewers,average_asin
21061,B000002VS5,"[2, 2]",5.0,This is both Graham Parker's best album and a ...,"03 22, 2008",A3KJ6JAZPH382D,"Tim Brough ""author and music buff""",Sparks create explosions,1206144000,True,3.976303,4.647059


In [33]:
# Make the DTM  
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(ngram_range=(1, 1),strip_accents='ascii',analyzer='word')
dtm_train = count_vect.fit_transform(train.reviewText)
dtm_evaluation = count_vect.transform(evaluation.reviewText)
dtm_testing =count_vect.transform(evaluation.reviewText)

In [34]:
#use TF-IDF weighting
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=True)
dtm_train = tf_transformer.fit_transform(dtm_train)
dtm_evaluation = count_vect.transform(evaluation.reviewText)
dtm_testing = count_vect.transform(testing.reviewText)

In [35]:
# Train SVM model
from sklearn.linear_model import SGDClassifier
clf_SVM=SGDClassifier(loss='hinge',alpha=0.001,learning_rate='optimal',class_weight={1: 1.2})
clf_SVM.fit(train[['average_reviewers','average_asin']], train.good)
# Predict the categories
predictions_SVM = clf_SVM.predict(np.nan_to_num(evaluation[['average_reviewers','average_asin']]))

In [36]:
# Show the results
print(pd.crosstab(predictions_SVM, evaluation.good, rownames=['predictions_SVM'], colnames=['real']))
print(metrics.classification_report(evaluation.good, predictions_SVM, target_names=["Bad", "Good"]))

real             False  True 
predictions_SVM              
False             2465   1141
True              1906   4194
             precision    recall  f1-score   support

        Bad       0.68      0.56      0.62      4371
       Good       0.69      0.79      0.73      5335

avg / total       0.69      0.69      0.68      9706



As the result shown, the precision is 0.68, and the f1-score is 0.68, which is a little lower than the improved naive bayes model.

## 2.4 Error analysis

In [37]:
b1="Allison Krauss has an amazing voice.  Her band is good too.  Easy listening and easy to hum or sing along."
b2="I like the Beach Boys, but I have to admit that I prefer their earlier California style hits (e.g., Fun, Fun, Fun) and I'm not a big fan of 60s music.  Nevertheless, \"Pet Sounds\" is undeniably a beautiful CD that still sounds great today.  \"Sloop John B\" and \"Wouldn't It Be Nice\" were top 10 hits in the US and \"God Only Knows\" and \"Caroline, No\" cracked the top 40, but all the songds sound great.  Brian Wilson's voice has never sounded more beautiful and the harmonies are near-perfect.  As a bonus, \"Hang on to Your Ego\" is included (it was recorded at the same time, but not released).  It's a great song and may be familiar to alt rock fans via a remake by Frank Black.  The liner notes are also terrific and include comments from Brian Wilson as well as a song by song description.  Great CD!"
b3="Allison Krauss has an amazing voice.  Her band is good too.  Easy listening and easy to hum or sing along."
b4="Alison Krauss has one of the most beautiful voice.  Her band is really good too.  Easy listening.  Easy to hum or sing along."
b5="Allison Krauss has a really amazing voice and sound.  Good band.  Easy listening, easy humming and easy to sing along."

In [48]:
print(clf_NB.predict(ch2.transform(tf_transformer.transform(count_vect.transform([b1])))))
print(clf_NB.predict(ch2.transform(tf_transformer.transform(count_vect.transform([b2])))))
print(clf_NB.predict(ch2.transform(tf_transformer.transform(count_vect.transform([b3])))))
print(clf_NB.predict(ch2.transform(tf_transformer.transform(count_vect.transform([b4])))))
print(clf_NB.predict(ch2.transform(tf_transformer.transform(count_vect.transform([b5])))))

[ True]
[ True]
[ True]
[ True]
[ True]


I choose 5 reviews from the training data, all of these 5 cases are predicted to be True as well as good, but their real catagoreis are bad.

# 3 Conclusion

To conduct sentiment analysis on the Amazon review data, by using different classifiers to predict the good and bad product reviews. The initial model provides a simple prediction on the good reviews, but this couldn't work on the bad ones. The Navie Bayes as the initial model, provides better prediction and f1-score, however, the difference between good and bad reviews is quite large. To improve the naive bayes model, dimension reduction works effectively on this case, and successfully reduces the difference between good and bad predictions. After that, by using grid search, I find the best parameter for SVM, and get the highest precision as 0.74. Finally, I personalize the prediction by using characteristics of the reviewer also using the SVM, the precision is 0.68, which is close to the initial model.

To conclude, if we want to get higher precision and f1-score, we could adjust the models by using different ways, such as TF-IDF weighting, changing the parameters, changing classifiers, so as to get a more precise prediction and acheive our goal.